# Organism

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Ensembl Species](https://useast.ensembl.org/info/about/species.html)
2. [NCBI Taxonomy](https://www.ncbi.nlm.nih.gov/taxonomy)

Here we show how to access and search Organism ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-organism --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
organisms = lb.Organism.public(organism="vertebrates")
organisms

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = organisms.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = organisms.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.giant_panda

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["giant panda"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = organisms.lookup(organisms.scientific_name)

In [ ]:
lookup.ailuropoda_melanoleuca

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
organisms.search("rabbit").head(3)

By default, search also covers synonyms:

In [ ]:
organisms.search("sapiens").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
organisms.search("sapiens", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
organisms.search(
    "oryctolagus_cuniculus",
    field=organisms.scientific_name,
).head()

## Standardize Organism identifiers

Let us generate a `DataFrame` that stores a number of Organism identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "spiny chromis"
        "silver-eye"
        "platyfish"
        "california sea lion"
        "This organism does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = organisms.validate(df_orig.index, organisms.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="Organism").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="ensembl", version="release-110", organism="vertebrates"
).one()
organisms= lb.Organism.public(public_source=public_source)
organisms

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-organism
!rm -r test-organism